In [1]:
from langchain_community.tools import ShellTool

shell_tool=ShellTool()

shell_tool.run({"commands":["echo 'hello world'"]})

Executing command:
 ["echo 'hello world'"]


c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


"'hello world'\r\n"

In [2]:
from langchain_core.tools import tool

@tool
def run_shell(command:str)->int:
    """在电脑上运行shell的命令操控电脑，例如输出command命令打开浏览器用百度搜索郭德纲，则commanda命令为 start chrome https://www.baidu.com/s?wd=郭德纲"""
    return shell_tool.run({"commands":[command]})
  
run_shell.invoke("mkdir test")

c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(
Error during command execution: 'utf-8' codec can't decode byte 0xd7 in position 0: invalid continuation byte


Executing command:
 ['mkdir test']


In [3]:
from langchain_openai import OpenAI,ChatOpenAI

llm=ChatOpenAI(
  model="qwen2.5-coder-1.5b-instruct",
  openai_api_key="EMPTY",
  base_url="http://127.0.0.1:1234/v1",
  temperature=0.7
)

In [4]:
@tool
def get_word_length(word:str)->int:
    """Return the length of a word."""
    return len(word)
  
get_word_length.invoke("hello")

5

In [5]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

@tool
def get_weather(location):
    """根据城市获取天气数据"""
    api_key = os.getenv("WEATHER_API_KEY")
    url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
    response=requests.get(url)
    print(location)
    if response.status_code==200:
        data=response.json()
        weather={
            'description':data['results'][0]['now']['text'],
            'temperature':data['results'][0]['now']['temperature']
        }
        return weather
    else:
        raise Exception(f"失败接收天气信息：{response.status_code}")

In [6]:
tools=[run_shell,get_word_length,get_weather]

In [7]:
promptTemplate = """尽可能的帮助用户回答任何问题。

您可以使用以下工具来帮忙解决问题，如果已经知道了答案，也可以直接回答：

{tools}

回复格式说明
----------------------------

回复我时，请以下列两种格式之一输出回复：

选项 1：如果您希望人类使用工具，请使用此选项。
采用以下JSON模式格式化的回复内容：

```json
{{
    "reason": string, \\ 叙述使用工具的原因
    "action": string, \\ 要使用的工具。必须是 {tool_names} 之一
    "action_input": string \\ 工具的输入
}}
````

选项2：如果您认为您已经有答案或者已经通过使用工具找到了答案，想直接对人类做出反应，请使用此选项。采用以下JSON模式格式化的回复内容：

```json
{{
  "action": "Final Answer",
  "answer": string \\ 最终答复问题的答案放到这里！
}}
````
用户的输入 -------------------- 
这是用户的输入（请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容）：

{input}
"""

In [8]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
      (
          "system",
          "你是一个非常有帮助的助手，你的任务是回答用户的问题。"
      ),
      (
          "human",
          promptTemplate
      ),
      MessagesPlaceholder(variable_name="agent_scratched")
    ]
)

In [9]:
from langchain_core.tools.render import render_text_description

prompt=prompt.partial(
    tools=render_text_description(list(tools)),
    tool_names=", ".join([t.name for t in tools])
)
prompt

ChatPromptTemplate(input_variables=['agent_scratched', 'input'], input_types={'agent_scratched': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annot

In [10]:
TEMPLATE_TOOL_RESPONSE="""
工具响应:
--------------------
{observation}

用户的输入:
--------------------
请根据工具的响应判断，是否能够回答问题:

{input}

请根据工具响应的内容，思考接下来回复。回复格式严格按照前面所说的2种JSON回复格式，选择其中1种进行回复。请记住通过单个选项，以JSON模式格式化的回复内容，不要回复其他内容。
"""

In [11]:
from langchain_core.messages import AIMessage,BaseMessage,HumanMessage

def format_log_to_messages(
    query,
    intermediate_steps,
    template_tool_response,
):
    """Construct the scratchpad that lets the agent continue its thought process."""
    thoughts:List[BaseMessage]=[]
    for action,observation in intermediate_steps:
        thoughts.append(AIMessage(content=action.log))
        human_message=HumanMessage(
           content=template_tool_response.format(
                observation=observation,
                input=query,
            )
        )
        thoughts.append(human_message)
    return thoughts

In [12]:
import logging
logger = logging.getLogger(__name__)
from langchain.agents.agent import AgentOutputParser
from langchain_core.utils.json import parse_json_markdown
from langchain_core.exceptions import OutputParserException
from langchain_core.agents import AgentAction, AgentFinish

class JSONAgentOutputParser(AgentOutputParser):
    """Parses tool invocations and final answers in JSON format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken, 
    should be in the below format. This will result in an AgentAction 
    being returned.

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```
    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish 
    being returned.
    ```
    {
    "action": "Final Answer",
    "answer": "4"
    }
    ```
    """
    
    def parse(self,text):
        try:
            response=parse_json_markdown(text)
            if isinstance(response,list):
                logger.warning("Got multiple action response:%s",response)
                response=response[0]
            if response["action"]=="Final Answer":
                return AgentFinish({"answer":response["answer"]},text)
            else:
                return AgentAction(
                    response["action"],response.get("action_input",{}),text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e
    
    @property
    def _type(self)->str:
        return "json-agent"

In [13]:
from langchain_core.runnables import Runnable,RunnablePassthrough

agent=(
    RunnablePassthrough.assign(
        agent_scratched=lambda x: format_log_to_messages(
            x["input"],
            x["intermediate_steps"],
            template_tool_response=TEMPLATE_TOOL_RESPONSE
        )
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

In [14]:
from langchain.agents import AgentExecutor

agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [15]:
agent_executor.invoke({"input":"创建一个test文件夹"})



> Entering new AgentExecutor chain...
 ```json
{
    "reason": "需要执行shell命令来创建文件夹",
    "action": "run_shell",
    "action_input": "mkdir test"
}
```Executing command:
 ['mkdir test']


c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 ```json
{
    "reason": "根据工具的输出判断是否能够回答问题",
    "action": "Final Answer",
    "answer": "可以创建一个名为 'test' 的文件夹。"
}
```

> Finished chain.


{'input': '创建一个test文件夹', 'answer': "可以创建一个名为 'test' 的文件夹。"}

In [17]:
agent_executor.invoke({"input":"用chrome打开https://www.bilibili.com"})



> Entering new AgentExecutor chain...
```json
{
    "reason": "Using run_shell tool to open a web browser in Chrome",
    "action": "run_shell",
    "action_input": "start chrome https://www.bilibili.com"
}
```Executing command:
 ['start chrome https://www.bilibili.com']


c:\Users\86151\anaconda3\envs\langchain_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


```json
{
  "action": "Final Answer",
  "answer": "I can't answer this question. Please try another tool or provide more context."
}
```

> Finished chain.


{'input': '用chrome打开https://www.bilibili.com',
 'answer': "I can't answer this question. Please try another tool or provide more context."}